In [1]:
import torch
import torch.optim as optim
import numpy as np
from transformer import Transformer
from metrics import Evaluator
from datasets import load_dataset
from utils import *
from training import *
from tokenizer import *

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device}")

/Users/tonimo/Documents/Developing/Repos/Deep-Learning/deep-learning-venv/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Using cpu


In [2]:
traindict = load_dataset("opus100", "de-en", split="train")
testdict = load_dataset("opus100", "de-en", split="test")

Found cached dataset opus100 (/Users/tonimo/.cache/huggingface/datasets/opus100/de-en/0.0.0/256f3196b69901fb0c79810ef468e2c4ed84fbd563719920b1ff1fdc750f7704)
Found cached dataset opus100 (/Users/tonimo/.cache/huggingface/datasets/opus100/de-en/0.0.0/256f3196b69901fb0c79810ef468e2c4ed84fbd563719920b1ff1fdc750f7704)


In [3]:
train_inputs, train_labels = get_split(traindict, "en", "de", size=100000)
test_inputs, test_labels = get_split(testdict, "en", "de", size=1000)
trainset = Dataset(train_inputs, train_labels)
testset = Dataset(test_inputs, test_labels)

In [4]:
trainframe = trainset.dataframe()
trainframe.head()

,inputs,labels
0,It's greed that it's gonna be the death of you...,Deine Habgier wird noch dein Tod sein.
1,Vega.,- Vega.
2,Just say when.,Sagen Sie einfach stopp.
3,- Wait.,- Warte.
4,I don't wanna be here.,Ich will nicht hier sein.


In [5]:
testframe = testset.dataframe()
testframe.head()

,inputs,labels
0,"By clicking on 'Save profile', you the user ag...",Die Nutzungsbedingungen werden durch das Klick...
1,I wanted to show you something first.,Ich wollte dir erst noch etwas zeigen.
2,You have suffered because of Shinkichi.,Du musstest wegen Shinkichi leiden.
3,"moodle:bg-bab: Calendar: Day view: Friday, 25 ...",moodle:bg-bab: Kalender: Tagesansicht: Freitag...
4,"I mean, most people, they see another person w...","Ich meine, die meisten Leuten sehen eine ander..."


In [6]:
print(trainframe.isnull().values.any())
trainframe.describe()

False


,inputs,labels
count,100000,100000
unique,96385,96634
top,Whereas:,in Erwägung nachstehender Gründe:
freq,49,48


In [7]:
print(testframe.isnull().values.any())
testframe.describe()

False


,inputs,labels
count,1000,1000
unique,999,998
top,Yes.,Ja.
freq,2,3


In [8]:
trainset.sample()

[("Galen says it's been here awhile.",
  'Galen hat gesagt, das liegt hier schon lange.')]

In [9]:
testset.sample()

[('This should be helpful for international guests. People with good knowledge in the English language will have noticed, that I sometimes have my own interpretation of English grammar ;-) Advices are welcomed!',
  'Dies soll vor allem internationalen Gästen zugute kommen. Leute mit guten Englischkenntnissen wird aufgefallen sein, dass ich der englischen Grammatik gelegentlich (gelinde gesagt) eine eigene Interpretation entgegenhalte ;-) Verbesserungsvorschläge sind daher jederzeit gerne erwünscht!')]

In [10]:
corpus = trainset.corpus() + testset.corpus()
tokenizer = Nerdimizer()
tokenizer.train(corpus, size=50000)

In [11]:
vocab_size = len(tokenizer)
maxlen = trainset.avglen(tokenizer, factor=20)
start, end, pad = tokenizer["[S]"], tokenizer["[E]"], tokenizer["[P]"]
tokenizer.padon(maxlen, pad_id=pad, end=True)
tokenizer.truncon(maxlen, end=True)
print(f"Number of word piece tokens: {vocab_size}\nMaxlen: {maxlen}")

AttributeError: 'Dataset' object has no attribute 'avg'

In [ ]:
tokenizedset = trainset.tokenized(tokenizer)
dataloader = tokenizedset.dataloader(batch_size=128, drop_last=False)

In [ ]:
model = Transformer(vocab_size, maxlen, pad_id=pad, dm=512, nhead=8, layers=6, dff=2048)
optimizer = optim.Adam(model.parameters(), lr=0.1, betas=(0.9, 0.98), eps=10e-9)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, factor=0.9, patience=10)
evaluator = Evaluator(testset, tokenizer, start, end, maxlen, sample=32, ngrams=4, threshold=20, 
                    mode="geometric", verbose=True, device=device)
checkpoint = Checkpoint(model, optimizer, scheduler, evaluator, epochs=25, 
                    path="checkpoints/english-german", overwrite=False, verbose=True)
model.to(device);

In [ ]:
# train(dataloader, model, optimizer, scheduler, evaluator, checkpoint, 
    # epochs=1000, warmups=100, verbose=True, device=device)